In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [2]:
from huggingface_hub import notebook_login
notebook_login()


In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [4]:
model="meta-llama/Llama-2-7b-chat-hf"

In [5]:
tokenizer=AutoTokenizer.from_pretrained(model,use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
model=AutoModelForCausalLM.from_pretrained(model,
                                           device_map='auto',
                                           torch_dtype=torch.float16,
                                           use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:

pipeline=transformers.pipeline("text-generation",
                               model=model,
                               tokenizer=tokenizer,
                               torch_dtype=torch.bfloat16,
                               device_map="auto",
                               # Maximum number of tokens to generate, a word is generally 2-3 tokens (minimum: 1)
                               max_new_tokens=512,
                               #Minimum number of tokens to generate. To disable this set it to -1. A word is generally 2-3 tokens (minimum 1)
                               min_new_tokens=-1,
                               #Adjusts the randomness of output
                               temperature=0.75,
                               do_sample=True,
                               top_k=30,
                               num_return_sequences=1,
                               eos_token_id=tokenizer.eos_token_id)

In [8]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [104]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [191]:
CUSTOM_SYSTEM_PROMPT="""
You are an advanced assistant that can extract entities from query.
If an user asks about finding length you can extract the word whose length needs to be found.
Suppose I have a query "find length of devesh", output should be ['length':'devesh']
If an user asks about multiplication you can extract the numbers which are required to be multiplied
suppose I have a query multiply two numbers 6 and 7, output should be ['1st number':6 ,'second number':7]
We are interested in extracting entities only as output
"""

In [192]:
instruction = "Extract enties:\n\n{text}"

In [193]:
SYSTEM_PROMPT=B_SYS + CUSTOM_SYSTEM_PROMPT + E_SYS

In [194]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [195]:
print(template)

[INST]<<SYS>>

You are an advanced assistant that can extract entities from query. 
If an user asks about finding length you can extract the word whose length needs to be found.
Suppose I have a query "find length of devesh", output should be ['length':'devesh']
If an user asks about multiplication you can extract the numbers which are required to be multiplied
suppose I have a query multiply two numbers 6 and 7, output should be ['1st number':6 ,'second number':7]
We are interested in extracting entities only as output

<<SYS>>

Extract enties:

{text}[/INST]


In [196]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [197]:
text ="what is the length of star. Also multiply numbers 5 and 10"

In [198]:
Chain=LLMChain(llm=llm, prompt=prompt)

In [199]:
res=Chain.run(text)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [131]:
print(type(res))

<class 'str'>


In [187]:
res

'[INST]<<SYS>>\n\nYou are an advanced assistant that can extract entities from query. \nIf an user asks about finding length you can extract the word whose length needs to be found.\nSuppose I have a query "find length of devesh", output should be [length:devesh]\nIf an user asks about multiplication you can extract the numbers which are required to be multiplied\nsuppose I have a query multiply two numbers 6 and 7, output should be [1st number:6] and [second number:7]\nWe are interested in extracting entities only as output\n\n<<SYS>>\n\nExtract enties:\n\nwhat is the length of star. Also multiply numbers 5 and 10[/INST]  Understood! I\'m ready to assist you. Please provide the queries, and I will extract the entities for you.\n\nQuery 1: What is the length of star?\nEntity: star\n\nQuery 2: Multiply numbers 5 and 10\nEntities:\n\n* 5\n* 10'

In [200]:
import re

def extract_entities(text):
    # Define a regular expression pattern to match text enclosed in brackets
    pattern = r"\[(.*?)\]"

    # Find all matches of the pattern in the text
    matches = re.findall(pattern, text)

    # Extract entities from matches
    entities = [match.split(':') for match in matches]


    return entities

# Example usage
entities = extract_entities(res)



In [201]:
entities

[['INST'],
 ["'length'", "'devesh'"],
 ["'1st number'", "6 ,'second number'", '7'],
 ['/INST'],
 ["'length'", "'star'"],
 ["'1st number'", " 5,'second number'", ' 10']]

In [202]:


# Find the index of '/INST'
end_index = None
for i, item in enumerate(entities):
    if item == ['/INST']:
        end_index = i
        break

# If '/INST' is found, extract text after it
if end_index is not None:
    text_after_inst = entities[end_index + 1:]
    print("Text after encountering '/INST':")
    print(text_after_inst)
else:
    print("'/INST' not found in the list.")


Text after encountering '/INST':
[["'length'", "'star'"], ["'1st number'", " 5,'second number'", ' 10']]


In [203]:

# Create a new list to store the modified data
modified_data = []

# Iterate through the data list and split elements containing commas
for sublist in text_after_inst:
    modified_sublist = []
    for item in sublist:
        if isinstance(item, str) and ',' in item:
            # Split the string by comma and add the elements separately
            split_items = item.split(',')
            modified_sublist.extend(split_items)
        else:
            modified_sublist.append(item)
    modified_data.append(modified_sublist)

print("Original data:")
print(entities)
print("\nModified data:")
print(modified_data)


Original data:
[['INST'], ["'length'", "'devesh'"], ["'1st number'", "6 ,'second number'", '7'], ['/INST'], ["'length'", "'star'"], ["'1st number'", " 5,'second number'", ' 10']]

Modified data:
[["'length'", "'star'"], ["'1st number'", ' 5', "'second number'", ' 10']]


In [205]:
modified_data[1][1]

' 5'

In [204]:
text1="find length of xyz"
res1=Chain.run(text1)
res1

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'[INST]<<SYS>>\n\nYou are an advanced assistant that can extract entities from query. \nIf an user asks about finding length you can extract the word whose length needs to be found.\nSuppose I have a query "find length of devesh", output should be [\'length\':\'devesh\']\nIf an user asks about multiplication you can extract the numbers which are required to be multiplied\nsuppose I have a query multiply two numbers 6 and 7, output should be [\'1st number\':6 ,\'second number\':7]\nWe are interested in extracting entities only as output\n\n<<SYS>>\n\nExtract enties:\n\nfind length of xyz[/INST]  Understood! In that case, when you provide a query like "find length of xyz", the output I can provide is:\n\n[\'entity\': \'xyz\']\n\nThis indicates that the entity being searched for is "xyz".'

In [206]:
entities = extract_entities(res1)

In [207]:
end_index = None
for i, item in enumerate(entities):
    if item == ['/INST']:
        end_index = i
        break

# If '/INST' is found, extract text after it
if end_index is not None:
    text_after_inst = entities[end_index + 1:]
    print("Text after encountering '/INST':")
    print(text_after_inst)
else:
    print("'/INST' not found in the list.")

Text after encountering '/INST':
[["'entity'", " 'xyz'"]]
